# CREATE MAIN CSV

This part create a new CSV file from all the data we collected during the scrapping part. These datas will be a bit more clean (lines with price = nan are dropped,...) but tthe data is still extremmely raw.

# CORRELATION RAW

This file plots a graph representing the correlation between the numerical data in the raw csv just created, we won't analyse it now.

# VARIABLE ON PRICE

This part annalyse the effect of every variables(except subtype of property) on the price, these files are "drafts",  going through them could be confusing, here is the main results I got from this first analysis (these conclusions will be used to re-clean the data)

In [2]:
#plt.savefig(r'.\images\num_facades.jpg')
#<img title="Effect of the number of facades on the price" alt="effect number facades" src="./variable_on_price/images/num_facades.jpg">

### Number of facade

<img title="Effect of the number of facades on the price" alt="effect number facades" src="./variable_on_price/images/num_facades.jpg">

We see that the graph has a strange beheviour for the number 5 and 6, after analysis I noticed that there are only 3 properties with these numbers. In consequence I will delete the lines where $$Number\_of\_facades > 4$$

### Number of rooms

<img title="Effect of the number of rooms on the price" alt="effect number rooms" src="./variable_on_price/images/num_rooms.jpg"> 

For number of rooms $\epsilon [0,7]$ we see progressive behaviour, after that it begins to be more strange. the numbers of properties wher number of rooms is greater than 7 is 224. In xonsequence I have decided to delete all lines where: $$numbers\_of\_rooms > 7 $$ 

### Type of Sale

98.8% of the type of sale are "residential sale", the remaining 1.2% is shared by 'annuity_without_lump_sum', 'annuity_monthly_amount' and 'first_session_with_reserve_price'. Those last 3 will be deleted, comparing monthly annuity with a global value has no logic. Residential sale being the only remaining value, this collumn will be deleted the make the end of the analyse easier

### Equipped kitchen

<img title="Effect of the type of kitchen on the price" alt="effect kitchen" src="./variable_on_price/images/kitchen.jpg">

The previous graph show us the effect of the kitchen on the price. We see that USA_UNINTALLED has a strange value as it is higher than USA_INSTALLED. This can be caused by the fact that there is only 7 properties with an USA_UNINSTALLED kitchen. It is not tthe only strange behaviour this graph has(see NAN), I wont go deeper in the analyse of it.

### Surface area of the plot of the land

<img title="surface area plot land in function of garden area" alt="plot land and garden area" src="./variable_on_price/images/plot_land.jpg">

We see there that the surface area of the plot of the land is highly correlated to the garden area, in consequence I will drop the collumn surface area of the plot of the land.

### making those changes:

In [6]:
import os
import warnings
import pandas as pd

In [7]:
#panda data frame containing every infos
col_names =  ["Locality",     
              "Type_of_property",
              "Subtype_of_property",
              "Price",
              "Type_of_sale",
              "Number_of_rooms",
              "Living_Area",
              "Fully_equipped_kitchen",
              "Furnished",
              "Open_fire",
              "Terrace",
              "Terrace_Area",
              "Garden",
              "Garden_Area",
              "Surface_area_of_the_plot_of_land",
              "Number_of_facades",
              "Swimming_pool",
              "State_of_the_building"
              ]
df_final  = pd.DataFrame(columns = col_names)
file_path = r"C:\Users\Sacha\Documents\BeCode\real-estate-price-prediction\data_analysis\main_csv.csv" 
df_final = pd.read_csv(file_path)

In [8]:
#keep only number of facades <=4
df_final = df_final[df_final["Number_of_facades"] < 5]

#keep only number of rooms <=7
df_final = df_final[df_final["Number_of_rooms"] < 8]

#delete the collumn type of sale
df_final = df_final[df_final["Type_of_sale"] == 'residential_sale']
del df_final["Type_of_sale"]

#delete the column equipped kitchen
del df_final["Fully_equipped_kitchen"]

#delete the column Surface_area_of_the_plot_of_land
del df_final["Surface_area_of_the_plot_of_land"]

#I will try to focus on comparing the living area and the price, that is why I will:
#delete the Furnished collumn which do not seem to have any interest, and the Subtype_of_property
del df_final["Furnished"]
del df_final["Subtype_of_property"]

#delete the collumns Garden and Terrace and writte 0 for all NAN in garden_area and terrace_area
del df_final["Garden"]
df_final["Garden_Area"] = df_final["Garden_Area"].fillna(0)

del df_final["Terrace"]
df_final["Terrace_Area"] = df_final["Terrace_Area"].fillna(0)

# and finally delete the lines where living area in NAN
df_final = df_final.dropna(subset=['Living_Area'])

#save the new CSV file I will work on
df_final.to_csv('analyse_csv.csv', index = False)

In [9]:
df_final.shape

(11717, 11)

# AREA ANALYSE

This part will show my analyse of the impact of the living area on the price, you can find all the codes in the area_analyse folder, and here are my conclusions:

* In the previous section I got this graph, this graph representing the price in function of the living area, it is hard to read and we do not see any links between the 2 variables.

<div style="text-align: center;">
<img title="Effect of the living area on the price" alt="effect living area" src="./variable_on_price/images/liv_area.jpg">
</div>

* This seems normal as differents localities have different prices per square metter as shown in the following graphs:

<img title="price per locality" alt="price per locality" src="./area_analyse/images/high_price.jpg">

<img title="low price per locality" alt="low price per locality" src="./area_analyse/images/low_price.jpg">

* __Firstly__ I want to notice that the __locality 8300__ has a price which is by far the greatest. It is also by far the locality which is the most represented in the data I should then be carefull that this locality does not introduce any errors.  
* __Then__, we see that the price per square meter can change by a factor 50 in function of the locality. 

I decided to plot the price in function of the living area for 10 localities, here is the result:
(fainted colour = confidence intervalle)

<div style="text-align: center;">
<img title="Effect of the number of facades on the price" alt="effect number facades" src="./area_analyse/images/liv_area_diff_loc.jpg">
</div>

We see that, at least for the 10 localities tested, there is a high correlation between the living area and the price. Obviously the problem here is to make any sort of conlusions on 10 localities while we have nearly 900 of them.
To be able to compare all those 900 localties I had to find a way to 'delete' this price per square meter, here is how I proceded:
* compute the median price per square meter for each localities
* delete the outliers of "Living area"
* for each properties compute the 'suppose living area' based on the property price and the price per square meter of its locality
* for each unique living area take one random line (this step is to make the graph more readable, keep it mind that the graph still doesn't represent the whole data)
* plot those 'suppose living area' in function of the real living area of the property
* you can find the code of this in the file supposed_are.ipynb

This will give us a graph where all localities can be represented. The more the dots (and the linear regression) will be close to the reference line y=x the more the Living_Area is a good estimator of the price (if we have the price per square meter of tthe locality).

<div style="text-align: center;">
<img title="supposed area in function of the real living area" alt="supposed area" src="./area_analyse/images/supposed_area.png">
</div>

For the data used the linear regression is close to the line y=x, however there is still a great variation of supposed living area in the squattered dots.
* dots __above__ the line y=x are properties which have less living area than they should have based on their price and their locality's price per square meter. It could be caused by a good state of the building, the presence of a swimming pool,...
* dots __under__ the line y=x are properties which have more living area than they should have based on their price and their locality's price per square meter

While I was testing the impact of each variable on the price, state of the building seemed to have a direct impact on the price of a property, as we can see on the following graph:
<div style="text-align: center;">
<img title="supposed area in function of the real living area" alt="supposed area" src="./variable_on_price/images/state_building.png">
</div>

It is now possible to 'merge' these 2 graphs together.

<div style="text-align: center;">
<img title="supposed area in function of the real living area with state of tthe building" alt="supposed area state building" src="./area_analyse/images/supposed_area_state_building.png">
</div>

### I will do now the same procedure with the price

<div style="text-align: center;">
<img title="supposed price in function of the real price" alt="supposed price" src="./area_analyse/images/supposed_price.png">
</div>

* dots __above__ the line y=x are properties which have a real price lower than expected, it can be caused by a huge propery which has to be renoved
* dots __under__ the line y=x are properties which havea real price higher than expected, it can be caused by a property which is as new,...

The linear regression has nearly the same slope as the reference line y=x. This is logic as we use tthe same data to plot the graph, compute the supposed price, and compute the prie per square meter. We should than use a sample of the data to computer the price(80% in my case) per square meter and than try this graph for the rest of the data. Here is the result:

<div style="text-align: center;">
<img title="price prediction" alt="price prediction" src="./area_analyse/images/prediction_price.png">
</div>